# Lesson 3: Reflection and Blogpost Writing

## Setup

In [4]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [5]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [6]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [7]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [8]:
print(reply)

Title: "Demystifying DeepLearning.AI: Your Guide to AI Mastery"

Unravel the world of artificial intelligence with DeepLearning.AI! Offering a plethora of courses created by AI expert Andrew Ng, this platform empowers you to delve into the realms of deep learning. From neural networks to computer vision, you'll gain hands-on experience in cutting-edge AI technologies. Whether you're a novice or a seasoned professional, DeepLearning.AI caters to all skill levels. Take the leap into the future of tech, sharpen your AI skills, and pave the way for groundbreaking innovations. Let DeepLearning.AI be your compass in navigating the exciting world of artificial intelligence!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [9]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [10]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unlocking the Power of AI with DeepLearning.AI"

Dive into the world of artificial intelligence with DeepLearning.AI! Offering cutting-edge courses created by industry experts like Andrew Ng, this platform equips you with the skills to thrive in the AI landscape. Whether you're a beginner or an AI enthusiast, there's something for everyone. From neural networks to natural language processing, you'll gain hands-on experience in the field. Join a global community of learners and take your first step towards a rewarding AI career. Explore the endless possibilities of AI and unleash your potential with DeepLearning.AI!

--------------------------------------------------------------------------------
Critic (to Writer):

The wr

## Nested chat

In [11]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [12]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [13]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [14]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [15]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [16]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [17]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unlocking the Power of AI with DeepLearning.AI"

Dive into the world of artificial intelligence with DeepLearning.AI! Offering cutting-edge courses created by industry experts like Andrew Ng, this platform equips you with the skills to thrive in the AI landscape. Whether you're a beginner or an AI enthusiast, there's something for everyone. From neural networks to natural language processing, you'll gain hands-on experience in the field. Join a global community of learners and take your first step towards a rewarding AI career. Explore the endless possibilities of AI and unleash your potential with DeepLearning.AI!

--------------------------------------------------------------------------------

**************************


--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of AI with DeepLearning.AI"

Embark on an AI journey with DeepLearning.AI! Immerse yourself in expert-led courses by industry stalwarts like Andrew Ng, honing skills essential for the AI realm. Whether a novice or an AI aficionado, there's content catering to all. From neural networks to natural language processing, practical knowledge awaits. Join a global cohort, kickstarting a fulfilling AI career. Explore AI's boundless horizons and unravel your potential with DeepLearning.AI!

--------------------------------------------------------------------------------


## Get the summary

In [ ]:
print(res.summary)